<a href="https://colab.research.google.com/github/shatisvigian/Big-Data-learning-Train-Scheduling/blob/main/xLSTM_modified_for_heart_sound_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import zipfile
import os

# Upload the dataset zip file
uploaded = files.upload()

# Extract the uploaded zip file
zip_file_name = list(uploaded.keys())[0]
zip_file_path = os.path.join('/content', zip_file_name)
extracted_path = '/content/extracted_data/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# List the contents of the extracted directory
extracted_contents = os.listdir(extracted_path)
print(extracted_contents)


Saving 2_H21_N_CAP_T1.wav to 2_H21_N_CAP_T1.wav
Saving H22N_CAP1T.wav to H22N_CAP1T.wav
Saving H22N_CAP3P.wav to H22N_CAP3P.wav
Saving H22N_CAP3T.wav to H22N_CAP3T.wav


BadZipFile: File is not a zip file

In [ ]:
import keras
from keras import backend as K
from keras.models import (Sequential, Model)
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf

# Assuming your prep module and get_train_test function are available

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x,y,batch_size=10):
    i = 0
    while(True):
        if i+batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i,i+batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i+batch_size
        x_batch = np.array(x_batch)
        yield x_batch,np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = y_train.shape[0]//batch_size
    validation_steps = y_test.shape[0]//batch_size

    train = generator(x_train,y_train,batch_size=batch_size)
    test = generator(x_test,y_test,batch_size=batch_size)

    if net_type == "basic":
        checkpointer = ModelCheckpoint(filepath='bestModelSCNN.hdf5', verbose=1, save_best_only=True)
    else:
        checkpointer = ModelCheckpoint(filepath='bestModelReSE.hdf5', verbose=1, save_best_only=True)
    out.fit_generator(train,steps_per_epoch=steps_per_epoch,epochs=10,validation_data=test,validation_steps=validation


SyntaxError: incomplete input (<ipython-input-7-724b12f5b2ad>, line 157)

In [ ]:
import keras
from keras import backend as K
from keras.models import (Sequential, Model)
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf

# Assuming your prep module and get_train_test function are available

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x,y,batch_size=10):
    i = 0
    while(True):
        if i+batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i,i+batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i+batch_size
        x_batch = np.array(x_batch)
        yield x_batch,np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = y_train.shape[0]//batch_size
    validation_steps = y_test.shape[0]//batch_size

    train = generator(x_train,y_train,batch_size=batch_size)
    test = generator(x_test,y_test,batch_size=batch_size)

    if net_type == "basic":
        checkpointer = ModelCheckpoint(filepath='bestModelSCNN.hdf5', verbose=1, save_best_only=True)
    else:
        checkpointer = ModelCheckpoint(filepath='bestModelReSE.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()


What model do you want to train? 
Type 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.
rese
Building a ReSE-2-Multi model.



NameError: name 'prep' is not defined

In [ ]:
from google.colab import files
import zipfile
import os

# Upload the dataset
uploaded = files.upload()

# Extract the dataset
for file_name in uploaded.keys():
    if zipfile.is_zipfile(file_name):
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall('/content/extracted_data')
        print("Extraction successful!")
    else:
        print(f"{file_name} is not a valid zip file.")


Saving a0007.zip to a0007 (1).zip
Extraction successful!


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()


In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x,y,batch_size=10):
    i = 0
    while(True):
        if i+batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i,i+batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i+batch_size
        x_batch = np.array(x_batch)
        yield x_batch,np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType


SyntaxError: unterminated string literal (detected at line 153) (<ipython-input-12-8d292a81fe12>, line 153)

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while(True):
        if i + batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i, i + batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = y_train.shape[0] // batch_size
    validation_steps = y_test.shape[0] // batch_size

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    if net_type == "basic":
        checkpointer = ModelCheckpoint(filepath='bestModelSCNN.hdf5', verbose=1, save_best_only=True)
    else:
        checkpointer = ModelCheckpoint(filepath='bestModelReSE.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
import zipfile
import os

# Upload the dataset zip file
uploaded = files.upload()

# Extract the uploaded zip file
zip_file_name = list(uploaded.keys())[0]
zip_file_path = os.path.join('/content', zip_file_name)
extracted_path = '/content/extracted_data/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# List the contents of the extracted directory
extracted_contents = os.listdir(extracted_path)
print(extracted_contents)


Saving healthy.zip to healthy.zip
['a0007.wav', 'healthy']


In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while(True):
        if i + batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i, i + batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = y_train.shape[0] // batch_size
    validation_steps = y_test.shape[0] // batch_size

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    if net_type == "basic":
        checkpointer = ModelCheckpoint(filepath='bestModelSCNN.hdf5', verbose=1, save_best_only=True)
    else:
        checkpointer = ModelCheckpoint(filepath='bestModelReSE.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()

What model do you want to train? 
Type 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.
rese
Building a ReSE-2-Multi model.



ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    if len(x_data) < 2:
        raise ValueError("Not enough data samples to perform train-test split.")

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while(True):
        if i + batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i, i + batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = max(1, y_train.shape[0] // batch_size)
    validation_steps = max(1, y_test.shape[0] // batch_size)

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    if net_type == "basic":
        checkpointer = ModelCheckpoint(filepath='bestModelSCNN.hdf5', verbose=1, save_best_only=True)
    else:
        checkpointer = ModelCheckpoint(filepath='bestModelReSE.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()


What model do you want to train? 
Type 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.
basic
Building a Sample CNN model.



ValueError: Not enough data samples to perform train-test split.

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    if len(x_data) < 2:
        print("Not enough data samples to perform train-test split.")
        return None, None, None, None

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while(True):
        if i + batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i, i + batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    if x_train is None or x_test is None or y_train is None or y_test is None:
        print("Insufficient data to proceed with training.")
        return

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = max(1, y_train.shape[0] // batch_size)
    validation_steps = max(1, y_test.shape[0] // batch_size)

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    if net_type == "basic":
        checkpointer = ModelCheckpoint(filepath='bestModelSCNN.hdf5', verbose=1, save_best_only=True)
    else:
        checkpointer = ModelCheckpoint(filepath='bestModelReSE.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()


What model do you want to train? 
Type 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.
rese
Building a ReSE-2-Multi model.

Not enough data samples to perform train-test split.
Insufficient data to proceed with training.


In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    if len(x_data) < 2:
        print("Not enough data samples to perform train-test split.")
        # Generate dummy data for testing
        num_samples = 10
        x_data = np.random.rand(num_samples, 59049, 1)
        y_data = np.random.randint(0, 12, num_samples)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while(True):
        if i + batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i, i + batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    if x_train is None or x_test is None or y_train is None or y_test is None:
        print("Insufficient data to proceed with training.")
        return

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = max(1, y_train.shape[0] // batch_size)
    validation_steps = max(1, y_test.shape[0] // batch_size)

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    if net_type == "basic":
        checkpointer =


SyntaxError: invalid syntax (<ipython-input-9-31e5f23d2f5f>, line 191)

In [ ]:
    Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while(True):
        if i + batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i, i + batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    if x_train is None or x_test is None or y_train is None or y_test is None:
        print("Insufficient data to proceed with training.")
        return

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = max(1, y_train.shape[0] // batch_size)
    validation_steps = max(1, y_test.shape[0] // batch_size)

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    checkpointer = ModelCheckpoint(filepath=f'bestModel{net_type.capitalize()}.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()


IndentationError: expected an indented block after function definition on line 12 (<ipython-input-11-9094ba8db63a>, line 14)

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    if len(x_data) < 2:
        print("Not enough data samples to perform train-test split.")
        # Generate dummy data for testing
        num_samples = 10
        x_data = np.random.rand(num_samples, 59049, 1)
        y_data = np.random.randint(0, 12, num_samples)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049, 1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese'):
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs=xc, outputs=x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while True:
        if i + batch_size >= y.shape[0]:
            i = 0
        x_batch = []
        y_batch = []
        for j in range(i, i + batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    if x_train is None or x_test is None or y_train is None or y_test is None:
        print("Insufficient data to proceed with training.")
        return

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = max(1, y_train.shape[0] // batch_size)
    validation_steps = max(1, y_test.shape[0] // batch_size)

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    checkpointer = ModelCheckpoint(filepath=f'bestModel{net_type.capitalize()}.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()


What model do you want to train? 
Type 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.
basic
Building a Sample CNN model.

Not enough data samples to perform train-test split.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 59049, 1)]        0         
                                                                 
 reshape (Reshape)           (None, 59049, 1)          0         
                                                                 
 conv1d (Conv1D)             (None, 19683, 128)        512       
                                                                 
 batch_normalization (Batch  (None, 19683, 128)        512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 19683, 128)        0       

IndexError: index 8 is out of bounds for axis 0 with size 8

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049,1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese') :
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs = xc, outputs = x)
    return model

def generator(x,y,batch_size=10):
    i = 0
    while(True):
        if i+batch_size >= y.shape[0]:
            i = 0
        x_batch = list()
        y_batch = list()
        for j in range(i,i+batch_size):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i+batch_size
        x_batch = np.array(x_batch)
        yield x_batch,np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType


SyntaxError: unterminated string literal (detected at line 153) (<ipython-input-13-8d292a81fe12>, line 153)

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    if len(x_data) < 2:
        print("Not enough data samples to perform train-test split.")
        # Generate dummy data for testing
        num_samples = 10
        x_data = np.random.rand(num_samples, 59049, 1)
        y_data = np.random.randint(0, 12, num_samples)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049, 1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese'):
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs=xc, outputs=x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while True:
        if i + batch_size > y.shape[0]:
            i = 0
        x_batch = []
        y_batch = []
        for j in range(i, min(i + batch_size, y.shape[0])):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    if x_train is None or x_test is None or y_train is None or y_test is None:
        print("Insufficient data to proceed with training.")
        return

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = max(1, y_train.shape[0] // batch_size)
    validation_steps = max(1, y_test.shape[0] // batch_size)

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    checkpointer = ModelCheckpoint(filepath=f'bestModel{net_type.capitalize()}.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()


What model do you want to train? 
Type 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.
basic
Building a Sample CNN model.

Not enough data samples to perform train-test split.
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 59049, 1)]        0         
                                                                 
 reshape_1 (Reshape)         (None, 59049, 1)          0         
                                                                 
 conv1d_10 (Conv1D)          (None, 19683, 128)        512       
                                                                 
 batch_normalization_11 (Ba  (None, 19683, 128)        512       
 tchNormalization)                                               
                                                                 
 activation_11 (Activation)  (None, 19683, 128)        0     

InvalidArgumentError: Graph execution error:

Detected at node mul_1 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-package

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    if len(x_data) < 2:
        print("Not enough data samples to perform train-test split.")
        # Generate dummy data for testing
        num_samples = 10
        x_data = np.random.rand(num_samples, 59049, 1)
        y_data = np.random.randint(0, 12, num_samples)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049, 1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese'):
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs=xc, outputs=x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while True:
        if i + batch_size > y.shape[0]:
            i = 0
        x_batch = []
        y_batch = []
        for j in range(i, min(i + batch_size, y.shape[0])):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    # Add debugging statements to check shapes
    print(f"y_true shape: {y_true.shape}, y_pred shape: {y_pred.shape}")

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    if x_train is None or x_test is None or y_train is None or y_test is None:
        print("Insufficient data to proceed with training.")
        return

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train, num_classes=12)  # Ensure y_train is one-hot encoded correctly
    y_test = to_categorical(y_test, num_classes=12)    # Ensure y_test is one-hot encoded correctly

    batch_size = 10
    steps_per_epoch = max(1, y_train.shape[0] // batch_size)
    validation_steps = max(1, y_test.shape[0] // batch_size)

    train = generator(x_train, y_train, batch_size=batch_size)
    test


In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.io.wavfile as swave
from skimage.transform import resize
import tensorflow as tf
import os

# Example implementation of the get_train_test function
def get_train_test():
    data_path = '/content/extracted_data/'  # Adjust this path as necessary
    x_data = []
    y_data = []

    # Example data loading logic
    for file_name in os.listdir(data_path):
        if file_name.endswith('.wav'):
            sample_rate, data = swave.read(os.path.join(data_path, file_name))
            data = resize(data, (59049, 1))  # Resize to desired input shape
            x_data.append(data)
            # Extract label from file_name or other logic
            label = int(file_name.split('a')[1].split('.')[0]) % 12  # Example label extraction logic
            y_data.append(label)

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    if len(x_data) < 2:
        print("Not enough data samples to perform train-test split.")
        # Generate dummy data for testing
        num_samples = 10
        x_data = np.random.rand(num_samples, 59049, 1)
        y_data = np.random.randint(0, 12, num_samples)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Define the prep module for simplicity
class prep:
    @staticmethod
    def get_train_test():
        return get_train_test()

def se_fn(x, amplifying_ratio):
    num_features = x.shape[-1]
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    return x

def basic_block(x, num_features, weight_decay, _):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
    if num_features != x.shape[-1]:
        shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                        kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    if amplifying_ratio > 0:
        x = Multiply()([x, se_fn(x, amplifying_ratio)])
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

def get_model(block_type='basic', multi=True, init_features=128, amplifying_ratio=16,
                drop_rate=0.5, weight_decay=0., num_classes=12):
    if block_type == 'rese':
        block = rese_block
    elif block_type == 'basic':
        block = basic_block
    else:
        raise Exception('Unknown block type: ' + block_type)

    xc = Input(shape=(59049, 1))
    x = Reshape([-1, 1])(xc)

    x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
                kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_features = init_features
    layer_outputs = []
    for i in range(9):
        num_features *= 2 if (i == 2 or i == 8) else 1
        x = block(x, num_features, weight_decay, amplifying_ratio)
        layer_outputs.append(x)

    if (multi) and (block_type == 'rese'):
        x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
    else:
        x = GlobalMaxPool1D()(x)

    x = Dense(x.shape[-1], kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if drop_rate > 0.:
        x = Dropout(drop_rate)(x)
    x = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)
    model = Model(inputs=xc, outputs=x)
    return model

def generator(x, y, batch_size=10):
    i = 0
    while True:
        if i + batch_size > y.shape[0]:
            i = 0
        x_batch = []
        y_batch = []
        for j in range(i, min(i + batch_size, y.shape[0])):
            x_batch.append(x[j])
            y_batch.append(y[j])
        i = i + batch_size
        x_batch = np.array(x_batch)
        yield x_batch, np.array(y_batch)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    # Add debugging statements to check shapes
    print(f"y_true shape: {y_true.shape}, y_pred shape: {y_pred.shape}")

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def main():
    net_type = input("What model do you want to train? \nType 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.\n")

    if net_type == "basic":
        print("Building a Sample CNN model.\n")
    elif net_type == "rese":
        print("Building a ReSE-2-Multi model.\n")
    else:
        raise Exception('Unknown model type: ' + net_type)

    x_train, x_test, y_train, y_test = prep.get_train_test()

    if x_train is None or x_test is None or y_train is None or y_test is None:
        print("Insufficient data to proceed with training.")
        return

    out = get_model(block_type=net_type)
    out.summary()

    out.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.categorical_crossentropy, metrics=['accuracy', f1])

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    batch_size = 10
    steps_per_epoch = max(1, y_train.shape[0] // batch_size)
    validation_steps = max(1, y_test.shape[0] // batch_size)

    train = generator(x_train, y_train, batch_size=batch_size)
    test = generator(x_test, y_test, batch_size=batch_size)

    checkpointer = ModelCheckpoint(filepath=f'bestModel{net_type.capitalize()}.hdf5', verbose=1, save_best_only=True)

    out.fit(train, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=test, validation_steps=validation_steps, callbacks=[checkpointer])

if __name__ == '__main__':
    main()

What model do you want to train? 
Type 'basic' for Sample CNN model and 'rese' for the ReSE-2-Multi model.
rese
Building a ReSE-2-Multi model.

Not enough data samples to perform train-test split.
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 59049, 1)]           0         []                            
                                                                                                  
 reshape_2 (Reshape)         (None, 59049, 1)             0         ['input_3[0][0]']             
                                                                                                  
 conv1d_20 (Conv1D)          (None, 19683, 128)           512       ['reshape_2[0][0]']           
                                                                                             

InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <la